# Obtención de datos

Para obtener los datos vamos a usar las descripciones en cada video de las mañaneras. Pues nos provee de un resumen del video del día. De esta manera ya no tenemos que lidiar con conversaciones del presidente cuando usamos las transcripciones.

## Importaciones

In [9]:
import json 
from datetime import datetime
import re

import pandas as pd 
import numpy as np

from googleapiclient.errors import HttpError
from Google import Create_Service

## Llamamiento a la API de YouTube

<code>retrieve_playlists_items</code> Función que hace la petición a la API de youtube. Se le da un objeto <code>service</code> que es la información sobre el tipo de API y la autenticación. También recibe <code>playlistId</code> que es el ID de la playlist que queremos.

In [10]:
def retrieve_playlists_items(service, playlistId):
    items =[]
    try:
        response = service.playlistItems().list(
            part = 'contentDetails, snippet, status',
            playlistId = playlistId,
            maxResults = 5
        ).execute()

        items.extend(response.get('items'))
        nextPageToken = response.get('nextPageToken')

        while nextPageToken:
            response = service.playlistItems().list(
                part = 'contentDetails, snippet, status',
                playlistId = playlistId,
                maxResults = 5,
                pageToken = nextPageToken
            ).execute()

            items.extend(response.get('items'))
            nextPageToken = response.get('nextPageToken')

        return items

    except HttpError as e:
        errMsg = json.loads(e.content)
        print("HTTP Error: ")
        print(errMsg['error']['message'])
        return []

Creando constantes para llamar a la API. Toda esta es información sobre el tipo de API y los archivos necesarios para hacer la conexión con Google Cloud

In [11]:
CLIENT_SECRETS_FILE = 'client_secret.json'  # Archivo .json que se obtiene al crear las credenciales OAuth en https://console.cloud.google.com/
API_NAME = 'youtube'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/youtube.readonly']

service = Create_Service(CLIENT_SECRETS_FILE, API_NAME, API_VERSION, SCOPES)

client_secret.json-youtube-v3-(['https://www.googleapis.com/auth/youtube.readonly'],)
['https://www.googleapis.com/auth/youtube.readonly']
youtube service created successfully


El objeto <code>service</code> nos da accesos a todos los recursos de la API

In [12]:
dir(service)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_basic_methods',
 '_add_nested_resources',
 '_add_next_methods',
 '_baseUrl',
 '_developerKey',
 '_dynamic_attrs',
 '_http',
 '_model',
 '_requestBuilder',
 '_resourceDesc',
 '_rootDesc',
 '_schema',
 '_set_dynamic_attr',
 '_set_service_methods',
 'abuseReports',
 'activities',
 'captions',
 'channelBanners',
 'channelSections',
 'channels',
 'close',
 'commentThreads',
 'comments',
 'i18nLanguages',
 'i18nRegions',
 'liveBroadcasts',
 'liveChatBans',
 'liveChatMessages',
 'liveChatModerators',
 'liveStreams',
 'members',
 'membershipsLevels',
 'new_batch_h

Hacemos una llamada a a función de <code>retrieve_playlists_items</code> para obtener todos los videos una playlist especifica. En este caso, los videos de la mañaneras

In [13]:
playlistId = 'PLRnlRGar-_296KTsVL0R6MEbpwJzD8ppA' # id de la playlist de las mañaneras
playlists_items = retrieve_playlists_items(service, playlistId)

In [14]:
playlists_items[0] # El último video agregado a la playlist

{'kind': 'youtube#playlistItem',
 'etag': 'X0Tqu-wq5dbVcs7e5vqC1zegOc0',
 'id': 'UExSbmxSR2FyLV8yOTZLVHNWTDBSNk1FYnB3SnpEOHBwQS45MDgzMzhDRDVDMzYwNDZE',
 'snippet': {'publishedAt': '2023-05-11T12:44:34Z',
  'channelId': 'UCxEgOKuI-n-WOJaNcisHvSg',
  'title': 'Sueldos de magistrados y jueces deben bajar para beneficiar al pueblo. Conferencia presidente AMLO',
  'description': "Conferencia de prensa matutina, desde Palacio Nacional. Jueves 11 de mayo 2023 | Presidente AMLO.\n\n0:00 Inicio de transmisión\n29:06 Comienza la conferencia de prensa del presidente Andrés Manuel López Obrador\n31:57 Fomento al deporte a través del Programa de Mejoramiento Urbano de la Sedatu\n46:13 Salvamento arqueológico en el proyecto del Tren Maya\n1:01:41 Sesión de preguntas y respuestas\n\nEn cuatro años de Gobierno de la Cuarta Transformación, a través del Programa de Mejoramiento Urbano, se han construido más espacios deportivos. La actividad física contribuye a tener una vida saludable y a reducir proble

## Guardando los datos en un DataFrame

Los datos están guardados en un json. Podemos obtener cada elemento de la variable utilizando las posiciones.

In [15]:
playlists_items[0]['snippet']

{'publishedAt': '2023-05-11T12:44:34Z',
 'channelId': 'UCxEgOKuI-n-WOJaNcisHvSg',
 'title': 'Sueldos de magistrados y jueces deben bajar para beneficiar al pueblo. Conferencia presidente AMLO',
 'description': "Conferencia de prensa matutina, desde Palacio Nacional. Jueves 11 de mayo 2023 | Presidente AMLO.\n\n0:00 Inicio de transmisión\n29:06 Comienza la conferencia de prensa del presidente Andrés Manuel López Obrador\n31:57 Fomento al deporte a través del Programa de Mejoramiento Urbano de la Sedatu\n46:13 Salvamento arqueológico en el proyecto del Tren Maya\n1:01:41 Sesión de preguntas y respuestas\n\nEn cuatro años de Gobierno de la Cuarta Transformación, a través del Programa de Mejoramiento Urbano, se han construido más espacios deportivos. La actividad física contribuye a tener una vida saludable y a reducir problemas sociales en las colonias.\xa0\n\nLa estrategia de fomento al deporte suma hasta el momento 896 espacios en el país en beneficio de más de 5 millones de personas. E

Extraeremos sólo la información necesaria para hacer el entrenamiento. Estos datos son: título, descripción y la fecha de publicación

In [16]:

data = []

for item in playlists_items:
    snippet = item['snippet']
    published_at = snippet['publishedAt']
    title = snippet['title']
    description = snippet['description']
    
    data.append([published_at, title, description])
    
df = pd.DataFrame(data, columns=['published_at', 'title', 'description'])
df = df.head(1087)

print(df)


              published_at                                              title   
0     2023-05-11T12:44:34Z  Sueldos de magistrados y jueces deben bajar pa...  \
1     2023-05-10T12:50:23Z  Homenaje a las madres de México en su día. Con...   
2     2023-05-09T12:47:41Z  Fin de la emergencia sanitaria por COVID-19 en...   
3     2023-05-08T12:45:24Z  Primer convoy del Tren Maya llegará el 8 de ju...   
4     2023-05-05T12:40:09Z  Alternativas legales para trabajar en Estados ...   
...                    ...                                                ...   
1082  2020-11-24T17:20:12Z  Presupuesto 2019 hará justicia social y dará c...   
1083  2019-03-08T21:59:25Z  Afores reducen comisiones. Fija Gobierno de Mé...   
1084  2019-03-08T21:59:25Z  Acuerdo de recompra de bonos del Aeropuerto ab...   
1085  2019-03-08T21:59:25Z  Gobierno reducirá gastos para garantizar presu...   
1086  2019-03-08T21:59:25Z  Devuelve parte de su salario el Presidente Lóp...   

                           

## Una primera limpieza

Los datos que llegan de la API están contaminados con información como minutos de YouTube en formato <code>00:00</code>, links, saltos de línea <code>\n</code> y palabras específicas como nombres redes sociales o frases que no aportan nada.
Para mejor ejemplo, tomemos un elemento.

In [17]:
df['description'][500]

"Conferencia de prensa matutina, desde Palacio Nacional. Martes 13 de abril 2021 | Presidente AMLO.\n\n0:00 Inicio de transmisión\n11:52 Comienza la conferencia de prensa del presidente Andrés Manuel López Obrador\n13:14 Pulso de la Salud\n32:48 Desarrollo de la vacuna mexicana 'Patria' contra COVID-19 \n48:23 Sesión de preguntas y respuestas\n2:23:42 Informe sobre la atención a menores migrantes no acompañados\n\nInformamos los avances del desarrollo de la vacuna mexicana 'Patria' contra COVID-19. Iniciarán los ensayos clínicos y, si todo sale bien, al final de 2021 se pondría a disposición de la Cofepris. Vamos hacia la soberanía en la producción de vacunas, un aspecto estratégico para el país.\n\nEn caso de tener éxito en las distintas fases de prueba, la vacuna permitirá ahorros económicos de hasta 855 por ciento respecto a la adquisición en el exterior.\n\nLa inversión hasta este día es de 150 millones de pesos, de los cuales, 135 millones de pesos provienen del Conacyt y 15 millo

Declararemos una serie de funciones que utilizarán regex para limpiar esta basura.

<code>replace_newlines(string)</code> es una función que elimina los patrones de salto de línea (<code>\n</code>) y coloca un espacio en blanco

In [18]:
def replace_newlines(string):
    pattern = r'\n'
    replace_with = ' '
    return re.sub(pattern, replace_with, string)

<code>replace_minutes(string)</code> es una función que elimina los patrones de formato de minuto (<code>00:00</code>)

In [19]:
def remove_minutes(string):
    pattern = r'[0-9]?[0-9]:[0-9][0-9]'
    replace_with = ''
    return re.sub(pattern, replace_with, string)

<code>remove_links(string)</code> es una función que elimina los patrones de los links de internet

In [20]:
def remove_links(string):
    pattern = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    replace_with = ''
    return re.sub(pattern, replace_with, string)

<code>remove_extra_words(string)</code> es una función que elimina los patrones de palabras específicas, pues estas sólo pueden afectar el entrenamiento

In [21]:
def remove_extra_words(string):
    pattern = r' Sigue las actividades del presidente de México:  Sitio web:   YouTube:   Facebook:   Twitter:   Instagram:   Telegram:    Spotify: '
    replace_with = ''
    return re.sub(pattern, replace_with, string)

Aplicamos las funciones y vemos los resultados

In [22]:
df['description'] = df['description'].apply(replace_newlines).apply(remove_minutes).apply(remove_links).apply(remove_extra_words)


In [23]:
df['description'][500]

"Conferencia de prensa matutina, desde Palacio Nacional. Martes 13 de abril 2021 | Presidente AMLO.   Inicio de transmisión  Comienza la conferencia de prensa del presidente Andrés Manuel López Obrador  Pulso de la Salud  Desarrollo de la vacuna mexicana 'Patria' contra COVID-19   Sesión de preguntas y respuestas :42 Informe sobre la atención a menores migrantes no acompañados  Informamos los avances del desarrollo de la vacuna mexicana 'Patria' contra COVID-19. Iniciarán los ensayos clínicos y, si todo sale bien, al final de 2021 se pondría a disposición de la Cofepris. Vamos hacia la soberanía en la producción de vacunas, un aspecto estratégico para el país.  En caso de tener éxito en las distintas fases de prueba, la vacuna permitirá ahorros económicos de hasta 855 por ciento respecto a la adquisición en el exterior.  La inversión hasta este día es de 150 millones de pesos, de los cuales, 135 millones de pesos provienen del Conacyt y 15 millones de pesos de la alianza establecida co

In [24]:
df

,published_at,title,description
0,2023-05-11T12:44:34Z,Sueldos de magistrados y jueces deben bajar pa...,"Conferencia de prensa matutina, desde Palacio ..."
1,2023-05-10T12:50:23Z,Homenaje a las madres de México en su día. Con...,"Conferencia de prensa matutina, desde Palacio ..."
2,2023-05-09T12:47:41Z,Fin de la emergencia sanitaria por COVID-19 en...,"Conferencia de prensa matutina, desde Palacio ..."
3,2023-05-08T12:45:24Z,Primer convoy del Tren Maya llegará el 8 de ju...,"Conferencia de prensa matutina, desde Palacio ..."
4,2023-05-05T12:40:09Z,Alternativas legales para trabajar en Estados ...,"Conferencia de prensa matutina, desde Puebla. ..."
...,...,...,...
1082,2020-11-24T17:20:12Z,Presupuesto 2019 hará justicia social y dará c...,Celebramos la aprobación del Presupuesto de Eg...
1083,2019-03-08T21:59:25Z,Afores reducen comisiones. Fija Gobierno de Mé...,Reiteramos el compromiso de cuidar los ahorros...
1084,2019-03-08T21:59:25Z,Acuerdo de recompra de bonos del Aeropuerto ab...,Celebramos el acuerdo que logró la SHCP para ...
1085,2019-03-08T21:59:25Z,Gobierno reducirá gastos para garantizar presu...,Hoy dimos a conocer que los gastos operativos ...


## Separación por semana

Ahora necesitamos separar por semana, es decir, juntar todas las descripciones de una semana. De esta manera cada renglón del dataframe reprensentará una semana en específico.
Lo primero que debemos hacer es limpiar el apartado de la fecha y convertirlo a datetime.

In [25]:
df['published_at']

0       2023-05-11T12:44:34Z
1       2023-05-10T12:50:23Z
2       2023-05-09T12:47:41Z
3       2023-05-08T12:45:24Z
4       2023-05-05T12:40:09Z
                ...         
1082    2020-11-24T17:20:12Z
1083    2019-03-08T21:59:25Z
1084    2019-03-08T21:59:25Z
1085    2019-03-08T21:59:25Z
1086    2019-03-08T21:59:25Z
Name: published_at, Length: 1087, dtype: object

Lo que se haces es: tomar los primeros 10 digitos de cada elemento de la columna y convertirlo al tipo datetime con la función <code>datetime.strptime</code>. Son los primeros 10 digitos porque son los que representan la fecha de ese día, no nos importan las horas, minutos o segundos así que lo descartamos

In [26]:
clean_dates = []

for dates in df['published_at']:
    clean_dates.append( datetime.strptime(dates[:10], '%Y-%m-%d') )
df['clean_dates'] = clean_dates

In [27]:
df = df.drop('published_at', axis=1) # Ahorra borramos las fechas que no nos sirven

In [28]:
df['clean_dates'] = pd.to_datetime(df['clean_dates'])

Ya tenemos los datos como fechas tipo <code>datetime</code>

In [29]:
df

,title,description,clean_dates
0,Sueldos de magistrados y jueces deben bajar pa...,"Conferencia de prensa matutina, desde Palacio ...",2023-05-11
1,Homenaje a las madres de México en su día. Con...,"Conferencia de prensa matutina, desde Palacio ...",2023-05-10
2,Fin de la emergencia sanitaria por COVID-19 en...,"Conferencia de prensa matutina, desde Palacio ...",2023-05-09
3,Primer convoy del Tren Maya llegará el 8 de ju...,"Conferencia de prensa matutina, desde Palacio ...",2023-05-08
4,Alternativas legales para trabajar en Estados ...,"Conferencia de prensa matutina, desde Puebla. ...",2023-05-05
...,...,...,...
1082,Presupuesto 2019 hará justicia social y dará c...,Celebramos la aprobación del Presupuesto de Eg...,2020-11-24
1083,Afores reducen comisiones. Fija Gobierno de Mé...,Reiteramos el compromiso de cuidar los ahorros...,2019-03-08
1084,Acuerdo de recompra de bonos del Aeropuerto ab...,Celebramos el acuerdo que logró la SHCP para ...,2019-03-08
1085,Gobierno reducirá gastos para garantizar presu...,Hoy dimos a conocer que los gastos operativos ...,2019-03-08


In [30]:
df['clean_dates']

0      2023-05-11
1      2023-05-10
2      2023-05-09
3      2023-05-08
4      2023-05-05
          ...    
1082   2020-11-24
1083   2019-03-08
1084   2019-03-08
1085   2019-03-08
1086   2019-03-08
Name: clean_dates, Length: 1087, dtype: datetime64[ns]

La mejor manera de manipular datos con fechas es creando una serie de tiempo, para estos se utiliza la libería <code>datetime</code>. Convertimos el Data Frame en una serie de tiempo con <code>pd.set_index</code>.
De esta manera las fechas ahora serán los índices de el dataframe

In [31]:
df.set_index('clean_dates', inplace=True)

In [32]:
df

,title,description
clean_dates,,
2023-05-11,Sueldos de magistrados y jueces deben bajar pa...,"Conferencia de prensa matutina, desde Palacio ..."
2023-05-10,Homenaje a las madres de México en su día. Con...,"Conferencia de prensa matutina, desde Palacio ..."
2023-05-09,Fin de la emergencia sanitaria por COVID-19 en...,"Conferencia de prensa matutina, desde Palacio ..."
2023-05-08,Primer convoy del Tren Maya llegará el 8 de ju...,"Conferencia de prensa matutina, desde Palacio ..."
2023-05-05,Alternativas legales para trabajar en Estados ...,"Conferencia de prensa matutina, desde Puebla. ..."
...,...,...
2020-11-24,Presupuesto 2019 hará justicia social y dará c...,Celebramos la aprobación del Presupuesto de Eg...
2019-03-08,Afores reducen comisiones. Fija Gobierno de Mé...,Reiteramos el compromiso de cuidar los ahorros...
2019-03-08,Acuerdo de recompra de bonos del Aeropuerto ab...,Celebramos el acuerdo que logró la SHCP para ...


Ahora le decimos que separe, con respecto a las fechas, semanalmente. De esta manera va a concatenar los contenidos de las descripciones y los títulos.

In [33]:
weekly_ts = df.resample('W').sum()

Se muestra el dataframe

In [34]:
weekly_ts

,title,description
clean_dates,,
2019-03-10,Presidente AMLO llama a terminar con feminicid...,Conferencia de prensa en vivo desde Palacio Na...
2019-03-17,Reformas constitucionales aprobadas representa...,Conferencia de prensa en vivo desde Palacio Na...
2019-03-24,Presidente firma compromiso de no reelección; ...,Conferencia de prensa en vivo desde Palacio Na...
2019-03-31,Acciones para mejorar protección a defensores ...,Conferencia de prensa en vivo desde Palacio Na...
2019-04-07,Avanza entrega de apoyos de 'Programas Integra...,"Conferencia de prensa matutina, desde Palacio ..."
...,...,...
2023-04-16,Avanza instalación de doble vía electrificada ...,"Conferencia de prensa matutina, desde Palacio ..."
2023-04-23,Avance mayor al 60 por ciento en obras del tra...,"Conferencia de prensa matutina, desde Palacio ..."
2023-04-30,Conferencia de prensa matutina desde Palacio N...,"Conferencia de prensa matutina, desde Palacio ..."


Ahora se guarda el dataframe en un .txt para que el preprocesamiento pueda llamarlo y utlizarlo

In [35]:
weekly_ts['description'].to_csv('mañaneras.csv')

Así es como se termina la obtención de datos, ya tenemos nuesto dataframe listo para mandar al preprocesamiento. Como se observó, a los datos se les hizo una primera limpieza, la razón de esto fue para que en el preprocesamiento se hicieran los pasos fundamentales y no pasos extras, como fue este caso de quitar ciertos patrones que estorbaban.